In [2]:
import os
import numpy as np
import torch
import pytorch3d
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

In [7]:
device = torch.device("cuda:0")
trg_obj = os.path.join('./data/dolphin.obj')
verts, faces, aux = load_obj(trg_obj)

faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
verts.shape, faces_idx.shape

(torch.Size([2562, 3]), torch.Size([5120, 3]))

In [9]:
center = verts.mean(0)
verts = verts - center
scale = max(verts.abs().max(0)[0])
verts = verts / scale

# We construct a Meshes structure for the target mesh
trg_mesh = Meshes(verts=[verts], faces=[faces_idx])
trg_mesh

In [11]:
src_mesh = ico_sphere(4, device)
src_mesh

In [12]:
mesh_edge_loss(src_mesh)

tensor(0.0057, device='cuda:0')

In [13]:
mesh_edge_loss(trg_mesh)

tensor(0.0020, device='cuda:0')

In [15]:
mesh_normal_consistency(src_mesh)

tensor(0.0010, device='cuda:0')

In [16]:
mesh_laplacian_smoothing(src_mesh, method="uniform")

tensor(0.0040, device='cuda:0')

In [20]:
verts[None].expand(3, -1, -1).shape, faces_idx[None].expand(3, -1, -1).shape

(torch.Size([3, 2562, 3]), torch.Size([3, 5120, 3]))

In [21]:
batch_mesh = Meshes(verts=verts[None].expand(3, -1, -1), faces=faces_idx[None].expand(3, -1, -1))
batch_mesh

In [22]:
mesh_edge_loss(batch_mesh)

tensor(0.0020, device='cuda:0')

In [6]:
import torch

torch.tensor(make_faces(4, 4)).dtype

torch.int64